# ACTIVIDAD DE CLASIFICACIÓN DE TEXTO

En esta actividad vamos a trabajar en clasificar textos. Se recorrerá todo el proceso desde traer el dataset hasta proceder a dicha clasificación. Durante la actividad se llevarán a cabo muchos procesos como la creación de un vocabulario, el uso de embeddings y la creación de modelos.

Las cuestiones presentes en esta actividad están basadas en un Notebook creado por François Chollet, uno de los creadores de Keras y autor del libro "Deep Learning with Python". 

En este Notebook se trabaja con el dataset "Newsgroup20" que contiene aproximadamente 20000 mensajes que pertenecen a 20 categorías diferentes.

El objetivo es entender los conceptos que se trabajan y ser capaz de hacer pequeñas experimentaciones para mejorar el Notebook creado.

#Librerías

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

ModuleNotFoundError: No module named 'numpy'

# Descarga de Datos

In [2]:
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True,
)

17329808/17329808 [==============================] - 16s 1us/step


In [3]:
import os
import pathlib

#Estructura de directorios del dataset
data_dir = pathlib.Path(data_path).parent / "20_newsgroup"
dirnames = os.listdir(data_dir)
print("Number of directories:", len(dirnames))
print("Directory names:", dirnames)

Number of directories: 20
Directory names: ['comp.windows.x', 'rec.sport.hockey', 'alt.atheism', 'sci.space', 'talk.politics.misc', 'sci.crypt', 'comp.sys.ibm.pc.hardware', 'rec.sport.baseball', 'sci.med', 'sci.electronics', 'talk.religion.misc', 'comp.graphics', 'comp.os.ms-windows.misc', 'rec.autos', 'talk.politics.mideast', 'comp.sys.mac.hardware', 'soc.religion.christian', 'talk.politics.guns', 'rec.motorcycles', 'misc.forsale']


In [4]:
#Algunos archivos de la categoria "com.graphics"
fnames = os.listdir(data_dir / "comp.graphics")
print("Number of files in comp.graphics:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of files in comp.graphics: 1000
Some example filenames: ['38341', '38512', '38619', '38821', '38864']


In [5]:
#Ejemplo de un texto de la categoría "com.graphics"
print(open(data_dir / "comp.graphics" / "39625").read())

Path: cantaloupe.srv.cs.cmu.edu!rochester!udel!gatech!howland.reston.ans.net!newsserver.jvnc.net!castor.hahnemann.edu!hal.hahnemann.edu!brennan
From: brennan@hal.hahnemann.edu
Newsgroups: comp.graphics
Subject: .GIFs on a Tek401x ??
Date: 15 MAY 93 14:29:54 EST
Organization: Hahnemann University
Lines: 14
Message-ID: <15MAY93.14295461@hal.hahnemann.edu>
NNTP-Posting-Host: hal.hahnemann.edu


      I was skimming through a few gophers and bumped into one at NIH
   with a database that included images in .GIF format.  While I have
   not yet worked out the kinks of getting the gopher client to call
   an X viewer, I figure that the majority of the users here are not
   in an X11 environment - instead using DOS and MS-Kermit.

      With Kermit supporting Tek4010 emulation for graphics display,
   does anyone know of a package that would allow a Tek to display a
   .GIF image?  It would be of more use to the local population to
   plug something of this sort in as the 'picture' command in

In [6]:
#Algunos archivos de la categoria "talk.politics.misc"
fnames = os.listdir(data_dir / "talk.politics.misc")
print("Number of files in talk.politics.misc:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of files in talk.politics.misc: 1000
Some example filenames: ['178736', '178951', '178528', '178388', '178677']


In [7]:
#Ejemplo de un texto de la categoría "talk.politics.misc"
print(open(data_dir / "talk.politics.misc" / "178463").read())

Xref: cantaloupe.srv.cs.cmu.edu talk.politics.guns:54219 talk.politics.misc:178463
Newsgroups: talk.politics.guns,talk.politics.misc
Path: cantaloupe.srv.cs.cmu.edu!magnesium.club.cc.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!magnus.acs.ohio-state.edu!usenet.ins.cwru.edu!agate!spool.mu.edu!darwin.sura.net!martha.utcc.utk.edu!FRANKENSTEIN.CE.UTK.EDU!VEAL
From: VEAL@utkvm1.utk.edu (David Veal)
Subject: Re: Proof of the Viability of Gun Control
Message-ID: <VEAL.749.735192116@utkvm1.utk.edu>
Lines: 21
Sender: usenet@martha.utcc.utk.edu (USENET News System)
Organization: University of Tennessee Division of Continuing Education
References: <1qpbqd$ntl@access.digex.net> <C5otvp.ItL@magpie.linknet.com>
Date: Mon, 19 Apr 1993 04:01:56 GMT

[alt.drugs and alt.conspiracy removed from newsgroups line.]

In article <C5otvp.ItL@magpie.linknet.com> neal@magpie.linknet.com (Neal) writes:

>   Once the National Guard has been called into federal service,
>it is under the command of the present. Tha N

In [8]:
samples = []
labels = []
class_names = []
class_index = 0
for dirname in sorted(os.listdir(data_dir)):
    class_names.append(dirname)
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
    print("Processing %s, %d files found" % (dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:]
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

Processing alt.atheism, 1000 files found
Processing comp.graphics, 1000 files found
Processing comp.os.ms-windows.misc, 1000 files found
Processing comp.sys.ibm.pc.hardware, 1000 files found
Processing comp.sys.mac.hardware, 1000 files found
Processing comp.windows.x, 1000 files found
Processing misc.forsale, 1000 files found
Processing rec.autos, 1000 files found
Processing rec.motorcycles, 1000 files found
Processing rec.sport.baseball, 1000 files found
Processing rec.sport.hockey, 1000 files found
Processing sci.crypt, 1000 files found
Processing sci.electronics, 1000 files found
Processing sci.med, 1000 files found
Processing sci.space, 1000 files found
Processing soc.religion.christian, 997 files found
Processing talk.politics.guns, 1000 files found
Processing talk.politics.mideast, 1000 files found
Processing talk.politics.misc, 1000 files found
Processing talk.religion.misc, 1000 files found
Classes: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.ha

# Mezclando los datos para separarlos en Traning y Test

In [9]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

¿Por qué mezclamos los datos antes de separarlos en entrenamiento y validación?

In [10]:
#Tu respuesta aqui

#Es necesario dividirlas para asegurarnos que las clases estén correctamente balanceadas tanto en la distribución que se usará para entrenamiento como para la de validación. 

# Tokenización de las palabras con TextVectorization 

In [11]:
from tensorflow.keras.layers import TextVectorization
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

In [12]:
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'the', 'to', 'of']

In [13]:
len(vectorizer.get_vocabulary())

20000

Pregunta. En la construcción del vocabulario hemos limitado el número de tokens ¿Podrías indicar el número de token diferentes o tamaño del vocabulario sin limitar el número de tokens? Es decir, ¿Cuántas palabras diferentes existen en los documentos procesados como instancias?

In [14]:
#Tu código aqui

#Para responder a la pregunta hay que hacer uso nuevo del método TextVectorization pero en esta ocasión no limitarle el número de tokens, es decir, eliminando donde indica max_tokens=20.000. A continuación se deja el código para resolver esta pregunta.
#vectorizer2 = TextVectorization(output_sequence_length=200)
#text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
#vectorizer2.adapt(text_ds)
#len(vectorizer2.get_vocabulary())

# Viendo la salida de Vectorizer

In [15]:
output = vectorizer([["the cat sat on the mat"]])
output.numpy()[0, :6]

array([   2, 3503, 1819,   15,    2, 6631])

In [16]:
output

<tf.Tensor: shape=(1, 200), dtype=int64, numpy=
array([[   2, 3503, 1819,   15,    2, 6631,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   

In [17]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [18]:
test = ["the", "cat", "sat", "on", "the", "mat"]
[word_index[w] for w in test]

[2, 3503, 1819, 15, 2, 6631]

Pregunta. La salida de vectorizer() para codificar los tokens ["El", "gato", "está", "sobre", "el", "tejado"] es la siguiente [1, 121, 405, 1, 45, 4561]. Si cada uno de los valores indica el índice en el que se encuentra cada palabra en el array creado para codificarla. ¿Podría ser correcta esta salida?

In [19]:
#Tu respuesta aqui

#No es correcta la salida porque como se puede ver el token “El” ha sido codificado con un 2 (es decir, con todo ceros salvo la un 1 en la posición 2) y el token “sobre” también ha sido codificado con un 2, es decir, exactamente igual que el token “El”. Cada “token” debe tener una codificación diferente. Para más error, se puede ver que “El” posteriormente tiene asignada otra codificación diferente 45 (es decir, un vector con todo ceros salvo un uno en la posición 45. Esto no es posible. 

# Tokenización de los datos de entrenamiento y validación

In [20]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

#Creación del modelo con un embedding hecho a mano con redes neuronales clásicas


In [21]:
modeloEmbeddingManual = keras.models.Sequential()
modeloEmbeddingManual.add(keras.layers.Embedding(20000, 10, input_length=200))
modeloEmbeddingManual.add(keras.layers.Flatten())
modeloEmbeddingManual.add(keras.layers.Dense(512, activation='relu'))
modeloEmbeddingManual.add(keras.layers.Dropout(0.3))
modeloEmbeddingManual.add(keras.layers.Dense(20, activation='softmax'))

In [22]:
modeloEmbeddingManual.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modeloEmbeddingManual.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"])
modeloEmbeddingManual.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))
print(modeloEmbeddingManual.summary())

Epoch 1/20
125/125 [==============================] - 3s 6ms/step - loss: 2.7085 - acc: 0.1351 - val_loss: 2.2827 - val_acc: 0.2481
Epoch 2/20
125/125 [==============================] - 1s 5ms/step - loss: 1.8392 - acc: 0.3988 - val_loss: 1.6616 - val_acc: 0.4394
Epoch 3/20
125/125 [==============================] - 1s 5ms/step - loss: 1.1948 - acc: 0.6315 - val_loss: 1.2928 - val_acc: 0.5529
Epoch 4/20
125/125 [==============================] - 1s 5ms/step - loss: 0.7652 - acc: 0.7747 - val_loss: 1.0896 - val_acc: 0.6294
Epoch 5/20
125/125 [==============================] - 1s 5ms/step - loss: 0.5014 - acc: 0.8599 - val_loss: 1.0398 - val_acc: 0.6564
Epoch 6/20
125/125 [==============================] - 1s 5ms/step - loss: 0.3471 - acc: 0.9027 - val_loss: 1.0170 - val_acc: 0.6784
Epoch 7/20
125/125 [==============================] - 1s 5ms/step - loss: 0.2533 - acc: 0.9277 - val_loss: 1.0385 - val_acc: 0.6882
Epoch 8/20
125/125 [==============================] - 1s 5ms/step - loss: 0.

#Evaluación

In [23]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = modeloEmbeddingManual(x)
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model.predict(
    [["this message is about computer graphics and 3D modeling"]]
)

class_names[np.argmax(probabilities[0])]

1/1 [==============================] - 0s 130ms/step


'comp.graphics'

In [24]:
probabilities = end_to_end_model.predict(
    [["we are talking about politics"]]
)

class_names[np.argmax(probabilities[0])]

1/1 [==============================] - 0s 29ms/step


'talk.politics.misc'

In [25]:
probabilities = end_to_end_model.predict(
    [["we are talking about religion"]]
)

class_names[np.argmax(probabilities[0])]

1/1 [==============================] - 0s 33ms/step


'talk.politics.misc'